## Contexte

Nous avons des **stations de mesure** (captor 1, captor 2, captor 3, etc.) qui envoient des relevés de capteurs sous la forme suivante :

```yaml
temp: Float 
sis: Float
hygro: Float
anem1: Float
anem2: Float
action: enum['A', 'B', 'C', 'SB']
À chaque relevé, la station décide d’une action (A, B, C ou SB pour “rien”) en fonction des valeurs de ses capteurs.

Problèmes constatés :

Parfois, l’action choisie n’est pas correcte.

Certaines actions sont manquantes (null).

Objectif :

Créer une solution de machine learning capable de :

Corriger les erreurs dans les données.

Compléter les valeurs manquantes des capteurs.

Prédire la bonne action pour chaque relevé.

Données disponibles :

Fichiers CSV contenant des relevés des différentes stations.

Certaines données incluent une validation manuelle de l’action (action_valide) pour l’entraînement.

Un fichier captor_3_todo.csv contient les relevés à corriger pour démontrer la solution.

Contraintes :

L’ingénieur d’origine est parti, et le code source des stations est perdu.

L’objectif principal est d’approcher 0 erreur sur les actions.

## 1️⃣ Exploration des données
- Comprendre la distribution des capteurs
- Vérifier valeurs manquantes et aberrantes
- Identifier patterns de la colonne `action`

## 2️⃣ Nettoyage et correction des données
- Remplacer ou imputer les valeurs manquantes
- Détecter et corriger les erreurs de relevé

## 3️⃣ Apprentissage d’un modèle pour prédire `action`
- Définir X et y
- Choisir modèle (Arbre de décision, Random Forest, XGBoost)
- Entraîner et évaluer le modèle


## 4️⃣ Complétion des relevés manquants (captor_2)
- Imputation des valeurs manquantes
- Prédiction des actions manquantes

## 5️⃣ Application sur `captor_3_todo.csv`
- Imputation → Prédiction → Export CSV
- Validation via l’API



## 6️⃣ Points d’optimisation
- Vérifier importance des variables
- Ajuster le modèle pour actions mal prédites
- Ajouter des règles métiers si nécessaire

### Import


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
captor_1 = pd.read_csv("/home/nabil_simplon/station-de-ski-machine-learning/data/captor_1_sample_1.csv")

In [11]:
display(captor_1)

,temp,sis,hygro,anem1,anem2,action
0,0.368054,8.644942,33.426743,3.678358,3.003935,C
1,0.473067,19.201138,36.650412,3.203428,2.596145,C
2,-1.186940,6.652969,29.483716,1.990443,1.595889,SB
3,0.004678,255.000000,28.666211,0.523551,0.419784,SB
4,-0.215133,3.445193,29.941774,3.029866,2.435523,A
...,...,...,...,...,...,...
995,1.123827,1.072103,41.172254,1.734741,1.389355,SB
996,-0.791947,1.556232,36.365359,2.812201,2.279837,A
997,0.501554,4.752910,36.125068,1.141020,0.927515,SB
998,2.328836,1.443901,60.176882,2.016030,1.645949,SB


In [12]:
captor_1.describe()

,temp,sis,hygro,anem1,anem2
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.015151,18.380897,36.017153,4.554538,2.702658
std,1.018223,38.956671,6.839586,25.214197,16.027085
min,-3.061247,0.019655,27.026040,0.000000,0.000000
25%,-0.656328,3.197099,31.048155,0.962532,0.863277
50%,-0.016409,6.680760,34.471957,2.032664,1.678843
75%,0.682131,14.976158,38.924882,3.042573,2.468254
max,3.255607,255.000000,68.521520,255.000000,255.000000
